## Running an Elasticsearch cluster
Uncomment the code below to run an Elasticsearch cluster.

In [13]:
# docker run -d \
#     --name elasticsearch \
#     -m 4GB \
#     -p 9200:9200 \
#     -p 9300:9300 \
#     -e "discovery.type=single-node" \
#     -e "xpack.security.enabled=false" \
#     docker.elastic.co/elasticsearch/elasticsearch:8.4.3

### Next Step: Confirm with an API Ping
To 100% confirm it's alive, run this in your terminal:

In [14]:
# !curl -X GET http://localhost:9200

### Q1. Running Elastic-Result

```json
{
  "name" : "1852f809dc58",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Zj0BSQXcS6CuWL28PaE0tA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
```

**What's the version.build_hash value?:** 42f05b9372a9a4a470db3b52817899b99a76ee73

In [1]:
import json
from openai import OpenAI
from elasticsearch import Elasticsearch

### Getting the data
Now let's get the FAQ data. You can run this snippet:

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
print(type(documents))

<class 'list'>


In [4]:
print(len(documents))

948


In [5]:
print(type(documents[0]))

<class 'dict'>


In [6]:
print(documents[0].keys())

dict_keys(['text', 'section', 'question', 'course'])


In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
documents[-1]

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

## Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

In [12]:
es_client = Elasticsearch('http://172.17.0.1:9200') 

In [13]:
es_client.info()

ObjectApiResponse({'name': 'bdbc40c79ada', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Zifa-ednRKWqZDb0XaFfFQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "fields": {
                    "keyword": { "type": "keyword" }
                }
            },
            "section": { "type": "text" },
            "question": {
                "type": "text",
                "fields": {
                    "keyword": { "type": "keyword" }
                }
            },
            "course": { "type": "keyword" }
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
from tqdm.auto import tqdm

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

**Which function do you use for adding your data to elastic?** <br>
**Answer:** index

### Q3. Searching
Now let's search in our index.<br>
We will execute a query "How do execute a command on a Kubernetes pod?".<br>
Use only question and text fields and give question a boost of 4, and use "type": "best_fields".<br>

What's the score for the top ranking result?<br>

- 84.50
- 64.50
- 44.50
- 24.50<br>
Look at the _score field.

In [19]:
#Question
query = 'How do execute a command on a Kubernetes pod?'

In [20]:
def elastic_search_top_score(query, filter=None, results=1):
    search_query = {
        "size": results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": filter
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit)
    
    return result_docs

In [21]:
elastic_search_top_score(query)

[{'_index': 'course-questions',
  '_id': 'HAWXZJcBm_zZyQBYRqOL',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}}]

In [22]:
print("The score of the top ranking result is: ", elastic_search_top_score(query)[0]['_score'])

The score of the top ranking result is:  44.50556


### Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".<br>

This time we are only interested in questions from machine-learning-zoomcamp.<br>

Return 3 results. What's the 3rd question returned by the search engine?<br>

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?<br>
- How do Lambda container images work?<br>
- How can I annotate a graph?

In [24]:
print("Question: What's the 3rd question returned by the search engine?")

filter = {
    "term": {
                "course": "machine-learning-zoomcamp"
            }
}
resul_size = 3
query = "How do copy a file to a Docker container?"

print("Answer: ", elastic_search_top_score(query, filter, resul_size )[2]['_source']['question'])

Question: What's the 3rd question returned by the search engine?
Answer:  How do I copy files from a different folder into docker container’s working directory?


### Question: What's the 3rd question returned by the search engine?
**Answer:**  How do I copy files from a different folder into docker container’s working directory?

### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do copy a file to a Docker container?" question to construct a prompt using the template below:

```python
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```


In [26]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

***What's the length of the resulting prompt? (use the len function)***
- 946
- 1446
- 1946
- 2446

In [27]:
records = []
results = elastic_search_top_score(query, filter, resul_size )
for record in results:
    records.append(record['_source'])

prompt = build_prompt(query, records)
print("The length of the prompt is: ", len(prompt))

The length of the prompt is:  1447


### Question: What's the length of the resulting prompt? (use the len function)
**Answer:** 1446

## Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

In [29]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tiktoken]


In [32]:
import tiktoken

# Load tokenizer for GPT-4
enc = tiktoken.encoding_for_model("gpt-4o")

tokens = enc.encode(prompt)

print("Token count:", len(tokens))
print("Token IDs:", tokens)

Token count: 321
Token IDs: [63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 5150, 261, 1974, 316, 261, 91238, 9282, 1715, 10637, 50738, 25, 793, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621

### Use the encode function. How many tokens does our prompt have?
**Answer:** 320

## Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?

In [35]:
def llm(prompt, client):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [39]:
def prompt_openAI(prompt):
    client = OpenAI()
    answer = llm(prompt, client)
    return answer

In [40]:
prompt_openAI(prompt)

'To copy a file from your local machine into a Docker container, you can use the `docker cp` command. The basic syntax is:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\n\nThis command allows you to specify the path to the file or directory on your local machine, the ID of the container, and the path within the container where you want the file to be copied.'

## Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

On June 17, the prices for gpt4o are:
- Input: $0.005 / 1K tokens

- Output: $0.015 / 1K tokens

In [45]:
avg_input_token = 150
avg_output_token = 250
requests = 1000

input_token = avg_input_token * requests
output_token = avg_output_token * requests

price_per_input_token = 0.005 / 1000
price_per_output_token = 0.015 / 1000

print(f"Price for {input_token} input token is: ", round(input_token* price_per_input_token, 2))
print(f"Price for {output_token} input token is: ", round(output_token* price_per_output_token, 2))

Price for 150000 input token is:  0.75
Price for 250000 input token is:  3.75
